In [23]:
#Habilitar si usas google colab

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
import pandas as pd

# Lectura de la data

In [24]:
# Utilizando google colab
#data = pd.read_csv('/content/drive/MyDrive/IA/PracticaCalificada2-CC421/train.csv', chunksize=42000000)

#Utilizando colab Franz
data = pd.read_csv('/content/drive/MyDrive/IA_PC2/train.csv', chunksize=1000000)

# Desde local (jupyter notebook)
#data = pd.read_csv('train.csv', chunksize=42000000)

In [25]:
data.get_chunk(5)

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


In [27]:
df = data.get_chunk() 

In [28]:
print(df.head())

                             key  ...  passenger_count
5    2011-01-06 09:50:45.0000002  ...                1
6    2012-11-20 20:35:00.0000001  ...                1
7   2012-01-04 17:22:00.00000081  ...                1
8  2012-12-03 13:10:00.000000125  ...                1
9   2009-09-02 01:11:00.00000083  ...                2

[5 rows x 8 columns]


# Preprocesamiento de la data

Se cuentan las filas con valores nulos, como estás representan una cantidad infíma
simplemente se eliminarán

In [29]:
print(df.isnull().sum())

key                   0
fare_amount           0
pickup_datetime       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude    10
dropoff_latitude     10
passenger_count       0
dtype: int64


In [30]:
df.dropna(inplace = True)
print(df.isnull().sum())

key                  0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
dtype: int64


In [31]:
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'], format='%Y-%m-%d %H:%M:%S UTC')

In [32]:
df['pickup_hour'] =df['pickup_datetime'].dt.hour

In [33]:
print(df['pickup_hour'])

5           9
6          20
7          17
8          13
9           1
           ..
1000000    21
1000001    16
1000002     8
1000003    17
1000004    12
Name: pickup_hour, Length: 999990, dtype: int64


In [34]:
print(df['passenger_count'].value_counts())

1      691344
2      148104
5       70595
3       43766
4       21429
6       21196
0        3555
208         1
Name: passenger_count, dtype: int64


In [35]:
df.loc[df.passenger_count > 7, 'passenger_count'] = 7

In [36]:
print(df['passenger_count'].value_counts())

1    691344
2    148104
5     70595
3     43766
4     21429
6     21196
0      3555
7         1
Name: passenger_count, dtype: int64


In [37]:
print(df.dtypes)
print(df.head())

key                          object
fare_amount                 float64
pickup_datetime      datetime64[ns]
pickup_longitude            float64
pickup_latitude             float64
dropoff_longitude           float64
dropoff_latitude            float64
passenger_count               int64
pickup_hour                   int64
dtype: object
                             key  fare_amount  ... passenger_count  pickup_hour
5    2011-01-06 09:50:45.0000002         12.1  ...               1            9
6    2012-11-20 20:35:00.0000001          7.5  ...               1           20
7   2012-01-04 17:22:00.00000081         16.5  ...               1           17
8  2012-12-03 13:10:00.000000125          9.0  ...               1           13
9   2009-09-02 01:11:00.00000083          8.9  ...               2            1

[5 rows x 9 columns]


In [38]:
df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_hour
5,2011-01-06 09:50:45.0000002,12.1,2011-01-06 09:50:45,-74.000964,40.731630,-73.972892,40.758233,1,9
6,2012-11-20 20:35:00.0000001,7.5,2012-11-20 20:35:00,-73.980002,40.751662,-73.973802,40.764842,1,20
7,2012-01-04 17:22:00.00000081,16.5,2012-01-04 17:22:00,-73.951300,40.774138,-73.990095,40.751048,1,17
8,2012-12-03 13:10:00.000000125,9.0,2012-12-03 13:10:00,-74.006462,40.726713,-73.993078,40.731628,1,13
9,2009-09-02 01:11:00.00000083,8.9,2009-09-02 01:11:00,-73.980658,40.733873,-73.991540,40.758138,2,1


In [39]:
df_nuevo = df.drop(['key','pickup_datetime'],1)

In [65]:
from math import cos, asin, sqrt, pi
def distancia(lat1, lon1, lat2, lon2):
    radioTierra = 6371
    p = math.pi/180
    a = 0.5 - cos((lat2 - lat1) * p)/2 + cos(lat1 * p) * cos(lat2 * p) * (1 - cos((lon2 - lon1) * p)) / 2
    return 2* radioTierra * asin(sqrt(a))

print(distancia(51.5,0,38.8,-77.1))

5918.185064088762


##Creando una columna distancia
Usamos la Haversine formula, que halla la distancia entre dos puntos en una esfera con longitud y latitud.
referencia: https://en.wikipedia.org/wiki/Haversine_formula

In [79]:
from math import cos, asin, sqrt, pi
p = math.pi/180
radioTierra = 6371
df["distancia_recorrida"] = df.apply(
    lambda fila: 2*radioTierra * asin(sqrt(0.5 - cos((fila.dropoff_latitude - fila.pickup_latitude) * p)/2 + cos(fila.pickup_latitude * p) * cos(fila.dropoff_latitude * p) * (1 - cos((fila.dropoff_longitude - fila.pickup_longitude) * p)) / 2)) 
, axis=1)

In [80]:
df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_hour,distancia_recorrida
5,2011-01-06 09:50:45.0000002,12.1,2011-01-06 09:50:45,-74.000964,40.731630,-73.972892,40.758233,1,9,3.787239
6,2012-11-20 20:35:00.0000001,7.5,2012-11-20 20:35:00,-73.980002,40.751662,-73.973802,40.764842,1,20,1.555807
7,2012-01-04 17:22:00.00000081,16.5,2012-01-04 17:22:00,-73.951300,40.774138,-73.990095,40.751048,1,17,4.155444
8,2012-12-03 13:10:00.000000125,9.0,2012-12-03 13:10:00,-74.006462,40.726713,-73.993078,40.731628,1,13,1.253232
9,2009-09-02 01:11:00.00000083,8.9,2009-09-02 01:11:00,-73.980658,40.733873,-73.991540,40.758138,2,1,2.849627


In [66]:
df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_hour
5,2011-01-06 09:50:45.0000002,12.1,2011-01-06 09:50:45,-74.000964,40.731630,-73.972892,40.758233,1,9
6,2012-11-20 20:35:00.0000001,7.5,2012-11-20 20:35:00,-73.980002,40.751662,-73.973802,40.764842,1,20
7,2012-01-04 17:22:00.00000081,16.5,2012-01-04 17:22:00,-73.951300,40.774138,-73.990095,40.751048,1,17
8,2012-12-03 13:10:00.000000125,9.0,2012-12-03 13:10:00,-74.006462,40.726713,-73.993078,40.731628,1,13
9,2009-09-02 01:11:00.00000083,8.9,2009-09-02 01:11:00,-73.980658,40.733873,-73.991540,40.758138,2,1


Se usarán las siguientes carácteristicas


In [19]:
X = df.drop(['key','pickup_datetime','fare_amount'],1)
y = df.fare_amount

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)